In [7]:
# Import modules
import ee
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pprint import pprint

In [4]:
# Authenticate
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
# Define geographic coordinates
lat = 39.186512 # This is y
lon = -96.576844 # This is x

# Convert coordinates into a Point geometry following the x,y notation
point = ee.Geometry.Point([lon, lat])

# Explore point geometry
point.getInfo()

In [ ]:
# Load digital elevation model (DEM) from Shuttle Radar Topography Mission (SRTM) 
dem = ee.Image('USGS/SRTMGL1_003')

In [ ]:
# Obtain some information about the DEM layer (output is long!)
pprint(dem.getInfo())

In [ ]:
# Retrieve the elevation value: This step will get us close to the answer, but we are not there yet
pprint(dem.sample(point, 1).getInfo())

In [ ]:
# Retrieve the elevation value: This step will get us even closer to the answer, but we are not there yet
dem.sample(point, 1).first().getInfo()

In [10]:
# Retrieve the elevation value: This step will get us the correct value
elev = dem.sample(point, 1).first().getNumber('elevation').getInfo()
print(f'Elevation: {elev} m')

Elevation: 317 m


In [5]:
# Load the SoilGrids dataset from GEE
soilgrids_sand = ee.Image("projects/soilgrids-isric/sand_mean")
soilgrids_clay = ee.Image("projects/soilgrids-isric/clay_mean")

# Load Polaris dataset
# Note: It's easier to load the image than the ImageCollection.
# Check the 'id' of each feature in the ImageCollection to get the link for the Image
# Code for ImageCollection: ee.ImageCollection('projects/sat-io/open-datasets/polaris/sand_mean')
polaris_sand = ee.Image('projects/sat-io/open-datasets/polaris/sand_mean/sand_0_5')
polaris_clay = ee.Image('projects/sat-io/open-datasets/polaris/clay_mean/clay_0_5')

In [ ]:
soilgrids_sand.getInfo()

In [8]:
# Create dictionary with station metadata

stations = [
    {'name': 'Ashland Bottoms', 'latitude': 39.125773, 'longitude': -96.63653},
    {'name': 'Belleville 2W', 'latitude': 39.81409, 'longitude': -97.675093},
    {'name': 'Colby', 'latitude': 39.39247, 'longitude': -101.06864},
    {'name': 'Garden City', 'latitude': 37.99733, 'longitude': -100.81514},
    {'name': 'Gypsum', 'latitude': 38.72522, 'longitude': -97.44415},
    {'name': 'Hutchinson 10SW', 'latitude': 37.93097, 'longitude': -98.02},
    {'name': 'Manhattan', 'latitude': 39.20857, 'longitude': -96.59169},
    {'name': 'Ottawa 2SE', 'latitude': 38.54268, 'longitude': -95.24647},
    {'name': 'Parsons', 'latitude': 37.36875, 'longitude': -95.28771},
    {'name': 'Rossville 2SE', 'latitude': 39.11661, 'longitude': -95.91572},
    {'name': 'Silver Lake 4E', 'latitude': 39.09213, 'longitude': -95.78153},
    {'name': 'Tribune 6NE', 'latitude': 38.53041, 'longitude': -101.66434},
    {'name': 'Woodson', 'latitude': 37.8612, 'longitude': -95.7836}
]

In [ ]:
# We will retrieve both datasets at 250-meter resolution to match the 
# coarser dataset.

# Iterate over each station metadata
for k,station in enumerate(stations):
    
    # Display current state of the loop
    print(f"Requesting data for {station['name']}")
    
    # Convert geographic coordinates into a Point geometry
    # following the x,y notation
    point = ee.Geometry.Point([station['longitude'], station['latitude']])
    
    # Soil Grids: Sample image and then select first and only feature with property value
    stations[k]['sand_soilgrids'] = soilgrids_sand.sample(point, 250).first().getNumber('sand_0-5cm_mean').multiply(0.1).getInfo()
    stations[k]['clay_soilgrids'] = soilgrids_clay.sample(point, 250).first().getNumber('clay_0-5cm_mean').multiply(0.1).getInfo()

    # Polaris: Sample image and then select first and only feature with property value
    stations[k]['sand_polaris'] = polaris_sand.sample(point, 250).first().getNumber('b1').getInfo()
    stations[k]['clay_polaris'] = polaris_clay.sample(point, 250).first().getNumber('b1').getInfo()

In [ ]:
# Convert dictionary into a dataframe for easier visualization
df = pd.DataFrame(stations)
df.head(3)

In [ ]:
plt.figure(figsize=(8,3))

plt.subplot(1,2,1)
plt.title('Sand 0-5 cm')
plt.scatter(df['sand_soilgrids'], df['sand_polaris'],facecolor='w',edgecolor='k')
plt.xlabel('Sand Soil Grids (%)')
plt.ylabel('Sand Polaris (%)')
plt.xlim([0,100])
plt.ylim([0,100])
plt.axline((0,0), slope=1, color='grey', linestyle=':')

plt.subplot(1,2,2)
plt.title('Clay 0-5 cm')
plt.scatter(df['clay_soilgrids'], df['clay_polaris'],facecolor='w',edgecolor='k')
plt.xlabel('Clay Soil Grids (%)')
plt.ylabel('Clay Polaris (%)')
plt.xlim([0,50])
plt.ylim([0,50])
plt.axline((0,0), slope=1, color='grey', linestyle=':')

plt.subplots_adjust(wspace=0.4)
plt.show()